In [ ]:
## Tutorial https://realpython.com/python-data-version-control/

Machine learning and data science come with a set of problems that are different from what you’ll find in traditional software engineering. Version control systems help developers manage changes to source code. But data version control, managing changes to models and datasets, isn’t so well established.

It’s not easy to keep track of all the data you use for experiments and the models you produce. Accurately reproducing experiments that you or others have done is a challenge. Many teams are actively developing tools and frameworks to solve these problems.

In this tutorial, you’ll learn how to:

    Use a tool called DVC to tackle some of these challenges
    Track and version your datasets and models
    Share a single development computer between teammates
    Create reproducible machine learning experiments


What Is DVC?

DVC is a command-line tool written in Python. It mimics Git commands and workflows to ensure that users can quickly incorporate it into their regular Git practice.

DVC is meant to be run alongside Git. In fact, the git and dvc commands will often be used in tandem, one after the other. While Git is used to store and version code, DVC does the same for data and model files. 

DVC uses a remote repository to store all your data and models. This is the single source of truth, and it can be shared amongst the whole team. You can get a local copy of the remote repository, modify the files, then upload your changes to share with team members.

The remote repository can be on the same computer you’re working on, or it can be in the cloud. DVC supports most major cloud providers, including AWS, GCP, and Azure. But you can set up a DVC remote repository on any server and connect it to your laptop. There are safeguards to keep members from corrupting or deleting the remote data.

When you store your data and models in the remote repository, a .dvc file is created. A .dvc file is a small text file that points to your actual data files in remote storage.

The .dvc file is lightweight and meant to be stored with your code in GitHub. When you download a Git repository, you also get the .dvc files. You can then use those files to get the data associated with that repository. Large data and model files go in your DVC remote storage, and small .dvc files that point to your data go in GitHub. 

There are six folders in your repository:

    src/ is for source code.
    data/ is for all versions of the dataset.
    data/raw/ is for data obtained from an external source.
    data/prepared/ is for data modified internally.
    model/ is for machine learning models.
    data/metrics/ is for tracking the performance metrics of your models.

The src/ folder contains three Python files:

    prepare.py contains code for preparing data for training.
    train.py contains code for training a machine learning model.
    evaluate.py contains code for evaluating the results of a machine learning model.


https://github.com/fastai/imagenette


Imagenette is a subset of the ImageNet dataset, which is often used as a benchmark dataset in many machine learning papers. ImageNet is too big to use as an example on a laptop, so you’ll use the smaller Imagenette dataset. Go to the Imagenette GitHub page and click 160 px download in the README.

Imagenette is a classification dataset, which means each image has an associated class that describes what’s in the image. To solve a classification problem, you need to train a model that can accurately determine the class of an image. It needs to look at an image and correctly identify what’s being shown.

The train/ and val/ folders are further divided into multiple folders. Each folder has a code that represents one of the 10 possible classes, and each image in this dataset belongs to one of ten classes:

    Tench
    English springer
    Cassette player
    Chain saw
    Church
    French horn
    Garbage truck
    Gas pump
    Golf ball
    Parachute

For simplicity and speed, you’ll train a model using only two of the ten classes, golf ball and parachute. When trained, the model will accept any image and tell you whether it’s an image of a golf ball or an image of a parachute. This kind of problem, in which a model decides between two kinds of objects, is called binary classification.

Copy the train/ and val/ folders and put them into your new repository, in the data/raw/ folder. Your repository structure should now look like this:

Practice the Basic DVC Workflow

The dataset you downloaded is enough to start practicing the DVC basics. In this section, you’ll see how DVC works in tandem with Git to manage your code and data.

To start, create a branch for your first experiment:

$ git checkout -b "first_experiment"

git checkout changes your current branch, and the -b switch tells Git that this branch doesn’t exist and should be created.

Next, you need to initialize DVC. Make sure you’re positioned in the top-level folder of the repository, then run dvc init:

$ dvc init

This will create a .dvc folder that holds configuration information, just like the .git folder for Git. In principle, you don’t ever need to open that folder, but you’ll take a peek in this tutorial so you can understand what’s happening under the hood.

Git gives you the ability to push your local code to a remote repository so that you have a single source of truth shared with other developers. Other people can check out your code and work on it locally without fear of corrupting the code for everyone else. The same is true for DVC.

You need some kind of remote storage for the data and model files controlled by DVC. This can be as simple as another folder on your system. Create a folder somewhere on your system outside the data-version-control/ repository and call it dvc_remote.

Now come back to your data-version-control/ repository and tell DVC where the remote storage is on your system:

$ dvc remote add -d remote_storage path/to/your/dvc_remote

DVC now knows where to back up your data and models. dvc remote add stores the location to your remote storage and names it remote_storage. You can choose another name if you want. The -d switch tells DVC that this is your default remote storage. You can add more than one storage location and switch between them.

You can always check what your repository’s remote is. Inside the .dvc folder of your repository is a file called config, which stores configuration information about the repository:


DVC supports many cloud-based storage systems, such as AWS S3 buckets, Google Cloud Storage, and Microsoft Azure Blob Storage. You can find out more in the official DVC documentation for the dvc remote add command.

Your repository is now initialized and ready for work. You’ll cover three basic actions:

    Tracking files
    Uploading files
    Downloading files

The basic rule of thumb you’ll follow is that small files go to GitHub, and large files go to DVC remote storage.

Tracking Files

Both Git and DVC use the add command to start tracking files. This puts the files under their respective control.

Add the train/ and val/ folders to DVC control:

$ dvc add data/raw/train

$ dvc add data/raw/val


Images are considered large files, especially if they’re collected into datasets with hundreds or thousands of files. The add command adds these two folders under DVC control. Here’s what DVC does under the hood:

    Adds your train/ and val/ folders to .gitignore
    Creates two files with the .dvc extension, train.dvc and val.dvc
    Copies the train/ and val/ folders to a staging area

This process is a bit complex and warrants a more detailed explanation.

.gitignore is a text file that has a list of files that Git should ignore, or not track. When a file is listed in .gitignore, it’s invisible to git commands. By adding the train/ and val/ folders to .gitignore, DVC makes sure you won’t accidentally upload large data files to GitHub.

You learned about .dvc files in the section What is DVC? They’re small text files that point DVC to your data in remote storage. Remember the rule of thumb: large data files and folders go into DVC remote storage, but the small .dvc files go into GitHub. When you come back to your work and check out all the code from GitHub, you’ll also get the .dvc files, which you can use to get your large data files.

Finally, DVC copies the data files to a staging area. The staging area is called a cache. When you initialized DVC with dvc init, it created a .dvc folder in your repository. In that folder, it created the cache folder, .dvc/cache. When you run dvc add, all the files are copied to .dvc/cache.

This raises two questions:

    Doesn’t copying files waste a lot of space?
    Can you put the cache somewhere else?

The answer to both questions is yes. You’ll work through both of these issues in the section Share a Development Machine.

Once the large image files have been put under DVC control, you can add all the code and small files to Git control with git add:

$ git add --all


To recap, large image files go under DVC control, and small files go under Git control. If someone wants to work on your project and use the train/ and val/ data, then they would first need to download your Git repository. They could then use the .dvc files to get the data.

But before people can get your repository and data, you need to upload your files to remote storage.

Uploading Files

To upload files to GitHub, you first need to create a snapshot of the current state of your repository. When you add all the modified files to the staging area with git add, create a snapshot with the commit command:

$ git commit -m "First commit with setup and DVC files"


DVC also has a commit command, but it doesn’t do the same thing as git commit. DVC doesn’t need a snapshot of the whole repository. It can just upload individual files as soon as they’re tracked with dvc add.

You use dvc commit when an already tracked file changes. If you make a local change to the data, then you would commit the change to the cache before uploading it to remote. You haven’t changed your data since it was added, so you can skip the commit step.

To upload your files from the cache to the remote, use the push command:

$ dvc push

DVC will look through all your repository folders to find .dvc files. As mentioned, these files will tell DVC what data needs to be backed up, and DVC will copy them from the cache to remote storage:

Your data is now safely stored in a location away from your repository. Finally, push the files under Git control to GitHub:

$ git push --set-upstream origin first_experiment

GitHub doesn’t know about the new branch you created locally, so the first push needs to use the --set-upstream option. origin is the place where your main, hosted version of the code is. In this case, it means GitHub. Your code and other small files are now safely stored in GitHub:

Well done! All your files have been backed up in remote storage.

Downloading Files

To learn how to download files, you first need to remove some of them from your repository.

As soon as you’ve added your data with dvc add and pushed it with dvc push, it’s backed up and safe. If you want to save space, you can remove the actual data. As long as all the files are tracked by DVC, and their .dvc files are in your repository, you can quickly get the data back.

You can remove the entire val/ folder, but make sure that the .dvc file doesn’t get removed:

$ rm -rf data/raw/val

To get your data back from the cache, use the dvc checkout command:

$ dvc checkout data/raw/val.dvc

Your data/raw/val/ folder has been restored. If you want DVC to search through your whole repository and check out everything that’s missing, then use dvc checkout with no additional arguments.

When you clone your GitHub repository on a new machine, the cache will be empty. The fetch command gets the contents of the remote storage into the cache:

$ dvc fetch data/raw/val.dvc

Or you can use just dvc fetch to get the data for all DVC files in the repository. Once the data is in your cache, check it out to the repository with dvc checkout. You can perform both fetch and checkout with a single command, dvc pull:

$ dvc pull

dvc pull executes dvc fetch followed by dvc checkout. It copies your data from the remote to the cache and into your repository in a single sweep. These commands roughly mimic what Git does, since Git also has fetch, checkout, and pull commands:

Keep in mind that you first need to get the .dvc files from Git, and only then can you call DVC commands like fetch and checkout to get your data. If the .dvc files aren’t in your repository, then DVC won’t know what data you want to fetch and check out.

You’ve now learned the basic workflow for DVC and Git. Whenever you add more data or change some code, you can add, commit, and push to keep everything versioned and safely backed up. For many people, this basic workflow will be enough for their everyday needs.

The rest of this tutorial focuses on some specific use cases like sharing computers with multiple people and creating reproducible pipelines. To explore how DVC handles these problems, you’ll need to have some code that runs machine learning experiments.

Build a Machine Learning Model

Using the Imagenette dataset, you’ll train a model to distinguish between images of golf balls and parachutes.

You’ll follow three main steps:

    Prepare data for training.
    Train your machine learning model.
    Evaluate the performance of your model.

As mentioned before, these steps correspond with the three Python files in the src/ folder:

    prepare.py
    train.py
    evaluate.py

The following subsections will explain what each file does. The entire file contents will be shown, followed by an explanation of what each line does.

Run the prepare.py script in the command line:

$ python src/prepare.py

When the script finishes, you’ll have the train.csv and test.csv files in your data/prepared/ folder. You’ll need to add these to DVC and the corresponding .dvc files to GitHub:

$ dvc add data/prepared/train.csv data/prepared/test.csv

$ git add --all

$ git commit -m "Created train and test CSV files"

Great! You now have a list of files to use for training and testing a machine learning model. The next step is to load the images and use them to run the training.

Now run the train.py script in the command line:

$ python src/train.py


When the script finishes, you’ll have a trained machine learning model saved in the model/ folder with the name model.joblib. This is the most important file of the experiment. It needs to be added to DVC, with the corresponding .dvc file committed to GitHub:

$ dvc add model/model.joblib

$ git add --all

$ git commit -m "Trained an SGD classifier"


Run the evaluate.py script in the command line:

$ python src/evaluate.py


Your model is now evaluated, and the metrics are safely stored in a the accuracy.json file. Whenever you change something about your model or use a different one, you can see if it’s improving by comparing it to this value.

In this case, your JSON file contains only one object, the accuracy of your model:

{ "accuracy": 0.670595690747782 }

If you print the accuracy variable multiplied by 100, you’ll get the percentage of correct classifications. In this case, the model classified 67.06 percent of test images correctly.

The accuracy JSON file is really small, and it’s useful to keep it in GitHub so you can quickly check how well each experiment performed:

$ git add --all

$ git commit -m "Evaluate the SGD model accuracy"

Great work! With evaluation completed, you’re ready to dig into some of DVC’s advanced features and processes.

Version Datasets and Models

At the core of reproducible data science is the ability to take snapshots of everything used to build a model. Every time you run an experiment, you want to know exactly what inputs went into the system and what outputs were created.

In this section, you’ll play with a more complex workflow for versioning your experiments. You’ll also open a .dvc file and look under the hood.

First, push all the changes you’ve made to the first_experiment branch to your GitHub and DVC remote storage:

$ git push

$ dvc push


Your code and model are now backed up on remote storage.

Training a model or finishing an experiment is a milestone for a project. You should have a way to find and return to this specific point.

Tagging Commits

A common practice is to use tagging to mark a specific point in your Git history as being important. Since you’ve completed an experiment and produced a new model, create a tag to signal to yourself and others that you have a ready-to-go model:

$ git tag -a sgd-classifier -m "SGDClassifier with accuracy 67.06%"

The -a switch is used to annotate your tag. You can make this as simple or as complex as you want. Some teams version their trained models with version number, like v1.0, v1.3, and so on. Others use dates and the initials of the team member who trained the model. You and your team decide how to keep track of your models. The -m switch allows you to add a message string to the tag, just like with commits.

Git tags aren’t pushed with regular commits, so they have to be pushed separately to your repository’s origin on GitHub or whatever platform you use. Use the --tags switch to push all tags from your local repository to the remote:

$ git push origin --tags


If you’re using GitHub, then you can access tags through the Releases tab of your repository.

You can always have a look at all the tags in the current repository:

$ git tag

DVC workflows heavily rely on effective Git practices. Tagging specific commits marks important milestones for your project. Another way to give your workflow more order and transparency is to use branching.

Creating One Git Branch Per Experiment

So far, you’ve done everything on the first_experiment branch. Complex problems or long-term projects often require running many experiments. A good idea is to create a new branch for every experiment.

In your first experiment, you set the maximum number of iterations of the model to 10. You can try setting that number higher to see if it improves the result. Create a new branch and call it sgd-100-iterations:

$ git checkout -b "sgd-100-iterations"

When you create a new branch, all the .dvc files you had in the previous branch will be present in the new branch, just like other files and folders.

Update the code in train.py so that the SGDClassifier model has the parameter max_iter=100:

That’s the only change you’ll make. Rerun the training and evaluation by running train.py and evaluate.py:

$ python src/train.py

$ python src/evaluate.py

You should now have a new model.joblib file and a new accuracy.json file.

Since the training process has changed the model.joblib file, you need to commit it to the DVC cache:

$ dvc commit

Remember, dvc commit works differently from git commit and is used to update an already tracked file. This won’t delete the previous model, but it will create a new one.

Add and commit the changes you’ve made to Git:

$ git add --all

$ git commit -m "Change SGD max_iter to 100"

Tag your new experiment:

$ git tag -a sgd-100-iter -m "Trained an SGD Classifier for 100 iterations"

$ git push origin --tags

Push the code changes to GitHub and the DVC changes to your remote storage:

$ git push --set-upstream origin sgd-100-iter

$ dvc push


You can jump between branches by checking out the code from GitHub and then checking out the data and model from DVC. For example, you can check out the first_example branch and get the associated data and model:

$ git checkout first_experiment

$ dvc checkout

Excellent. You now have multiple experiments and their results versioned and stored, and you can access them by checking out the content via Git and DVC.

Share a Development Machine

In many academic and work settings, computationally heavy work isn’t done on individual laptops because they’re not powerful enough to handle large amounts of data or intense processing. Instead, teams use cloud computers or on-premises workstations. Multiple users often work on a single machine.

With multiple users working with the same data, you don’t want to have many copies of the same data spread out among users and repositories. To save space, DVC allows you to set up a shared cache. When you initialize a DVC repository with dvc init, DVC will put the cache in the repository’s .dvc/cache folder by default.

Create Reproducible Pipelines

Here’s a recap of the steps you made so far to train your machine learning model:

    Fetching the data
    Preparing the data
    Running training
    Evaluating the training run

You fetched the data manually and added it to remote storage. You can now get it with dvc checkout or dvc pull. The other steps were executed by running various Python files. These can be chained together into a single execution called a DVC pipeline that requires only one command.

Create a new branch and call it sgd-pipeline:

$ git checkout -b sgd-pipeline

You’ll use this branch to rerun the experiment as a DVC pipeline. A pipeline consists of multiple stages and is executed using a dvc run command. Each stage has three components:

    Inputs
    Outputs
    Command

DVC uses the term dependencies for inputs and outs for outputs. The command can be anything you usually run in the command line, including Python files. You can practice creating a pipeline of stages while running another experiment. Each of your three Python files, prepare.py, train.py, and evaluate.py will be represented by a stage in the pipeline.

You’re going to reproduce all the files you created with prepare.py, train.py, and evaluate.py by creating pipelines.

A pipeline automatically adds newly created files to DVC control, just as if you’ve typed dvc add. Since you’ve manually added a lot of your files to DVC control already, DVC will get confused if you try to create the same files using a pipeline.

To avoid that, first remove the CSV files, models, and metrics using dvc remove:

$ dvc remove data/prepared/train.csv.dvc \
             data/prepared/test.csv.dvc \
             model/model.joblib.dvc --outs

This will remove the .dvc files and the associated data targeted by the .dvc files. You should now have a blank slate to re-create these files using DVC pipelines.

First, you’re going to run prepare.py as a DVC pipeline stage. The command for this is dvc run, which needs to know the dependencies, outputs, and command:

    Dependencies: prepare.py and the data in data/raw
    Outputs: train.csv and test.csv
    Command: python prepare.py

Execute prepare.py as a DVC pipeline stage with the dvc run command:

$ dvc run -n prepare \

        -d src/prepare.py -d data/raw \

        -o data/prepared/train.csv -o data/prepared/test.csv \

        python src/prepare.py

All this is a single command. The first row starts the dvc run command and accepts a few options:

    The -n switch gives the stage a name.
    The -d switch passes the dependencies to the command.
    The -o switch defines the outputs of the command.

The main argument to the command is the Python command that will be executed, python src/prepare.py. In plain English, the above dvc run command gives DVC the following information:

    Line 1: You want to run a pipeline stage and call it prepare.
    Line 2: The pipeline needs the prepare.py file and the data/raw folder.
    Line 3: The pipeline will produce the train.csv and test.csv files.
    Line 4: The command to execute is python src/prepare.py.

Once you create the stage, DVC will create two files, dvc.yaml and dvc.lock. Open them and take a look inside.

Great—you’ve automated the first stage of the pipeline, which can be visualized as a flow diagram:

You’ll use the CSV files produced by this stage in the following stage.

The next stage in the pipeline is training. The dependencies are the train.py file itself and the train.csv file in data/prepared. The only output is the model.joblib file. To create a pipeline stage out of train.py, execute it with dvc run, specifying the correct dependencies and outputs:

$ dvc run -n train \

        -d src/train.py -d data/prepared/train.csv \

        -o model/model.joblib \
        
        python src/train.py

Two down, one to go! The final stage will be the evaluation. The dependencies are the evaluate.py file and the model file generated in the previous stage. The output is the metrics file, accuracy.json. Execute evaluate.py with dvc run:

$ dvc run -n evaluate \

        -d src/evaluate.py -d model/model.joblib \

        -M metrics/accuracy.json \
        
        python src/evaluate.py

Notice that you used the -M switch instead of -o. DVC treats metrics differently from other outputs. When you run this command, it will generate the accuracy.json file, but DVC will know that it’s a metric used to measure the performance of the model.

You can get DVC to show you all the metrics it knows about with the dvc show command:

$ dvc metrics show

metrics/accuracy.json:

            accuracy: 0.6996197718631179

You’ve completed the final stage of the pipeline, which looks like this:

You can now see your entire workflow in a single image. Don’t forget to tag your new branch and push all the changes to GitHub and DVC:

$ git add --all

$ git commit -m "Rerun SGD as pipeline"

$ dvc commit

$ git push --set-upstream origin sgd-pipeline

$ git tag -a sgd-pipeline -m "Trained SGD as DVC pipeline."

$ git push origin --tags

$ dvc push


This will version and store your code, models, and data for the new DVC pipeline.

Now for the best part!

For training, you’ll use a random forest classifier, which is a different model that can be used for classification. It’s more complex than the SGDClassifier and could potentially yield better results. Start by creating and checking out a new branch and calling it random_forest:

$ git checkout -b "random_forest"

The power of pipelines is the ability to reproduce them with minimal hassle whenever you change anything. Modify your train.py to use a RandomForestClassifier instead of the SGDClassifier:

The only lines that changed were importing the RandomForestClassifier instead of the SGDClassifier, creating an instance of the classifier, and calling its fit() method. Everything else remained the same.

Since your train.py file changed, its MD5 hash has changed. DVC will realize that one of the pipeline stages needs to be reproduced. You can check what changed with the dvc status command:

$ dvc status

train:

    changed deps:
    
        modified:           src/train.py


This will display all the changed dependencies for every stage of the pipeline. Since the change in the model will affect the metric as well, you want to reproduce the whole chain. You can reproduce any DVC pipeline file with the dvc repro command:

$ dvc repro evaluate

And that’s it! When you run the repro command, DVC checks all the dependencies of the entire pipeline to determine what’s changed and which commands need to be executed again. Think about what this means. You can jump from branch to branch and reproduce any experiment with a single command!

To wrap up, push your random forest classifier code to GitHub and the model to DVC:

$ git add --all

$ git commit -m "Train Random Forrest classifier"

$ dvc commit

$ git push --set-upstream origin random_forest

$ git tag -a random_forest -m "Random Forest classifier with 80.99% accuracy."

$ git push origin --tags

$ dvc push

Now you can compare metrics across multiple branches and tags.

Call dvc metrics show with the -T switch to display metrics across multiple tags:

$ dvc metrics show -T

sgd-pipeline:

    metrics/accuracy.json:

        accuracy: 0.6996197718631179

forest:

    metrics/accuracy.json:
    
        accuracy: 0.8098859315589354


Awesome! This gives you a quick way to keep track of what the best-performing experiment was in your repository.

When you come back to this project in six months and don’t remember the details, you can check which setup was the most successful with dvc metrics show -T and reproduce it with dvc repro! Anyone else who wants to reproduce your work can do the same. They’ll just need to take three steps:

    Run git clone or git checkout to get the code and .dvc files.
    Get the training data with dvc checkout.
    Reproduce the entire workflow with dvc repro evaluate.

If they can write a script to fetch the data and create a pipeline stage for it, then they won’t even need step 2.

Nice work! You ran multiple experiments and safely versioned and backed up the data and models. What’s more, you can quickly reproduce each experiment by just getting the necessary code and data and executing a single dvc repro command.

Next Steps

Congratulations on completing the tutorial! You’ve learned how to use data version control in your daily work. If you want to go deeper into optimizing your workflow or learning more about DVC, then this section offers some suggestions.

Remembering to run all the DVC and Git commands at the right time can be a challenge, especially when you’re just getting started. DVC offers the possibility to integrate the two tighter together. Using Git hooks, you can execute DVC commands automatically when you run certain Git commands. Read more about installing Git hooks for DVC in the official docs.

Git and GitHub allow you to track the history of changes for a particular repository. You can see who updated what and when. You can create pull requests to update data. Patterns like these are possible with DVC as well. Have a look at the section on data registries in the DVC docs.

DVC even has a Python API, which means you can call DVC commands in your Python code to access data or models stored in DVC repositories.

Even though this tutorial provides a broad overview of the possibilities of DVC, it’s impossible to cover everything in a single document. You can explore DVC in detail by checking out the official User Guide, Command Reference, and Interactive Tutorials.
